# In this notebook, we will use the depolarization ratio to quality control a volume of data from the new radar at Radisson, Saskatchewan
## We will also visualize the data using some openly-available colour tables

In [ ]:
%pylab inline
import _raveio
import ropo_realtime, ec_drqc
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import GmapColorMap

### Block of look-ups for display

In [ ]:
PALETTE = {} # To be populated

UNDETECT = {"TH":GmapColorMap.PUREWHITE,
            "DBZH":GmapColorMap.PUREWHITE,
            "DR":GmapColorMap.PUREWHITE,
            "VRADH":GmapColorMap.GREY5,
            "RHOHV":GmapColorMap.PUREWHITE,
            "ZDR":GmapColorMap.PUREWHITE}

NODATA = {"TH":GmapColorMap.WEBSAFEGREY,
          "DBZH":GmapColorMap.WEBSAFEGREY,
          "DR":GmapColorMap.WEBSAFEGREY,
          "VRADH":GmapColorMap.GREY8,
          "RHOHV":GmapColorMap.WEBSAFEGREY,
          "ZDR":GmapColorMap.WEBSAFEGREY}

LEGEND = {"TH":'Radar reflectivity factor (dBZ)',
          "DBZH":'Radar reflectivity factor (dBZ)',
          "DR":'Depolarization ratio (dB)',
          "VRADH":'Radial wind velocity away from radar (m/s)',
          "RHOHV":'Cross-polar correlation coefficient',
          "ZDR":"Differential reflectivity (dB)"}

TICKS = {"TH":range(-30,80,10),
         "DBZH":range(-30,80,10),
         "ZDR":range(-8,9,2),
         "RHOHV":np.arange(0,11,1)/10.,
         "VRADH":range(-48,56,8),
         "DR":range(-36,3,3)}

### Colormap loader and loads

In [ ]:
def loadPal(fstr, reverse=True):
    fd = open(fstr)
    LINES = fd.readlines()
    fd.close()
    pal = []
    for line in LINES:
        s = line.split()
        if reverse: s.reverse()
        for val in s:
            pal.append(int(float(val)*255))
    if reverse: pal.reverse()
    return pal

# Colour maps by Fabio Crameri, http://www.fabiocrameri.ch/colourmaps.php, a couple of them modified
PALETTE["DBZH"] = loadPal("data/hawaii.txt")
PALETTE["DR"] = loadPal("data/moleron.txt", False)  # Modified oleron
PALETTE["ZDR"] = loadPal("data/oleron.txt", False)
PALETTE["RHOHV"] = loadPal("data/mroma.txt")        # Modified roma
PALETTE["VRADH"] = loadPal("data/vik.txt", False)

### Set up the display

In [ ]:
def display(obj):
    fig = plt.figure()
    default_size = fig.get_size_inches()
    fig.set_size_inches((default_size[0]*2, default_size[1]*2))

    paramname = obj.getParameterNames()[0]
    pal = PALETTE[paramname]
    pal[0], pal[1], pal[2] = UNDETECT[paramname]     # Special value - areas radiated but void of echo
    pal[767], pal[766], pal[765] = NODATA[paramname] # Special value - areas unradiated
    if paramname == "VRADH":
        pal[379], pal[380], pal[381] = GmapColorMap.PUREWHITE  # VRADH isodop
        pal[382], pal[383], pal[384] = GmapColorMap.PUREWHITE  # VRADH isodop
        pal[385], pal[386], pal[387] = GmapColorMap.PUREWHITE  # VRADH isodop
    colorlist = []
    for i in range(0, len(pal), 3):
        colorlist.append([pal[i]/255.0, pal[i+1]/255.0, pal[i+2]/255.0])

    param = obj.getParameter(paramname)
    data = param.getData()
    data = data*param.gain + param.offset
        
    im = plt.imshow(data,cmap=matplotlib.colors.ListedColormap(colorlist))
    cax = plt.gca()
    cax.axes.get_xaxis().set_visible(False)
    cax.axes.get_yaxis().set_visible(False)

    cb = plt.colorbar(ticks=TICKS[paramname])
    cb.set_label(LEGEND[paramname])
 
    plt.show()

## Do the science
### Read the polar volume, QC the reflectivity using legacy ROPO, and then save the QC:ed result

In [ ]:
rio = _raveio.open('data/2019051509_00_ODIMH5_PVOL6S_VOL_casra.16.h5')
rio.object = ropo_realtime.generate(rio.object)
rio.save('data/2019051509_00_ODIMH5_PVOL6S_VOL_casra.ropo.h5')

### Re-read the polar volume, QC it using depolarization ratio, and then save the QC:ed result

In [ ]:
rio = _raveio.open('data/2019051509_00_ODIMH5_PVOL6S_VOL_casra.16.h5')
pvol = rio.object
ec_drqc.drQC(pvol)
rio.object = pvol
rio.save('data/2019051509_00_ODIMH5_PVOL6S_VOL_casra.drqc.h5')

## NOW LEAVE THIS NOTEBOOK AND GO TO A TERMINAL to generate CAPPIs

### Read and display CAPPIs, starting with Doppler-corrected reflectivity

In [ ]:
cappi = _raveio.open('data/cappi_DBZH.h5').object
display(cappi)

### Differential reflectivity

In [ ]:
cappi = _raveio.open('data/cappi_ZDR.h5').object
display(cappi)

### Cross-polar correlation coefficient

In [ ]:
cappi = _raveio.open('data/cappi_RHOHV.h5').object
display(cappi)

### Radial wind velocity, lowest PPI

In [ ]:
ppi = _raveio.open('data/ppi_VRADH.h5').object
display(ppi)

### Depolarization ratio

In [ ]:
cappi = _raveio.open('data/cappi_DR.h5').object
display(cappi)

### ROPO:ed reflectivity

In [ ]:
cappi = _raveio.open('data/cappi_DBZH_ropo.h5').object
display(cappi)

### DRQC:ed reflectivity

In [ ]:
cappi = _raveio.open('data/cappi_DBZH_drqc.h5').object
display(cappi)